# Transport and Logistics Analysis Notebook

This notebook analyzes various aspects of a transport and logistics company's operations using the `TransportAndLogistics` database schema. Each section corresponds to a specific category of questions, with SQL queries and Python visualizations to derive insights.

In [ ]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine

# Database connection setup (replace with actual credentials)
db_username = "your_username"
db_password = "your_password"
db_host = "localhost"
db_port = "3306"
db_name = "TransportAndLogistics"

connection_string = f"mysql+pymysql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(connection_string)

# Helper function to execute SQL queries
def execute_query(query):
    with engine.connect() as conn:
        return pd.read_sql(query, conn)

## Section 1: Delivery Performance Analysis

### Question 1: Average delivery time for shipments across different regions

In [ ]:
query_avg_delivery_time = """
SELECT 
    s.RouteID,
    r.OriginCity,
    r.DestinationCity,
    AVG(DATEDIFF(s.DeliveryDate, s.ShipmentDate)) AS AvgDeliveryTime
FROM Shipments s
JOIN Routes r ON s.RouteID = r.RouteID
WHERE s.Status = 'Delivered'
GROUP BY s.RouteID, r.OriginCity, r.DestinationCity;
"""
avg_delivery_time = execute_query(query_avg_delivery_time)

# Visualization
plt.figure(figsize=(10, 6))
sns.barplot(x="AvgDeliveryTime", y="DestinationCity", data=avg_delivery_time.sort_values(by="AvgDeliveryTime", ascending=False))
plt.title("Average Delivery Time by Destination City")
plt.xlabel("Average Delivery Time (Days)")
plt.ylabel("Destination City")
plt.show()

### Question 2: Routes experiencing the most delays and contributing factors

In [ ]:
query_delayed_routes = """
SELECT 
    s.RouteID,
    r.OriginCity,
    r.DestinationCity,
    COUNT(*) AS DelayCount
FROM Shipments s
JOIN Routes r ON s.RouteID = r.RouteID
WHERE DATEDIFF(s.DeliveryDate, s.ShipmentDate) > r.EstimatedDuration
GROUP BY s.RouteID, r.OriginCity, r.DestinationCity
ORDER BY DelayCount DESC
LIMIT 10;
"""
delayed_routes = execute_query(query_delayed_routes)

# Visualization
plt.figure(figsize=(10, 6))
sns.barplot(x="DelayCount", y="DestinationCity", data=delayed_routes)
plt.title("Top Routes with Most Delays")
plt.xlabel("Number of Delays")
plt.ylabel("Destination City")
plt.show()

### Question 3: On-time delivery percentage

In [ ]:
query_on_time_delivery = """
SELECT 
    MONTH(s.ShipmentDate) AS Month,
    COUNT(CASE WHEN DATEDIFF(s.DeliveryDate, s.ShipmentDate) <= r.EstimatedDuration THEN 1 END) * 100.0 / COUNT(*) AS OnTimePercentage
FROM Shipments s
JOIN Routes r ON s.RouteID = r.RouteID
GROUP BY MONTH(s.ShipmentDate)
ORDER BY Month;
"""
on_time_delivery = execute_query(query_on_time_delivery)

# Visualization
plt.figure(figsize=(10, 6))
sns.lineplot(x="Month", y="OnTimePercentage", data=on_time_delivery, marker='o')
plt.title("On-Time Delivery Percentage by Month")
plt.xlabel("Month")
plt.ylabel("On-Time Delivery Percentage (%)")
plt.xticks(ticks=range(1, 13), labels=["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"])
plt.show()

### Question 4: Seasonal patterns in delivery delays

In [ ]:
query_seasonal_delays = """
SELECT 
    MONTH(s.ShipmentDate) AS Month,
    COUNT(*) AS DelayCount
FROM Shipments s
JOIN Routes r ON s.RouteID = r.RouteID
WHERE DATEDIFF(s.DeliveryDate, s.ShipmentDate) > r.EstimatedDuration
GROUP BY MONTH(s.ShipmentDate)
ORDER BY Month;
"""
seasonal_delays = execute_query(query_seasonal_delays)

# Visualization
plt.figure(figsize=(10, 6))
sns.barplot(x="Month", y="DelayCount", data=seasonal_delays)
plt.title("Seasonal Patterns in Delivery Delays")
plt.xlabel("Month")
plt.ylabel("Number of Delays")
plt.xticks(ticks=range(1, 13), labels=["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"])
plt.show()

## Section 2: Fleet Management and Resource Optimization

### Question 1: Underutilized vs overused vehicles

In [ ]:
query_vehicle_utilization = """
SELECT 
    v.VehicleID,
    v.VehicleType,
    COUNT(s.ShipmentID) AS ShipmentCount
FROM Vehicles v
LEFT JOIN Shipments s ON v.VehicleID = s.VehicleID
GROUP BY v.VehicleID, v.VehicleType;
"""
vehicle_utilization = execute_query(query_vehicle_utilization)

# Visualization
plt.figure(figsize=(10, 6))
sns.barplot(x="ShipmentCount", y="VehicleType", data=vehicle_utilization.sort_values(by="ShipmentCount", ascending=False))
plt.title("Vehicle Utilization by Type")
plt.xlabel("Number of Shipments")
plt.ylabel("Vehicle Type")
plt.show()

### Question 2: Fuel consumption rate per vehicle type

In [ ]:
query_fuel_consumption = """
SELECT 
    v.VehicleType,
    AVG(fc.FuelFilled / r.Distance) AS FuelConsumptionRate
FROM Vehicles v
JOIN FuelConsumptionRecords fc ON v.VehicleID = fc.VehicleID
JOIN Routes r ON v.VehicleID = r.RouteID
GROUP BY v.VehicleType;
"""
fuel_consumption = execute_query(query_fuel_consumption)

# Visualization
plt.figure(figsize=(10, 6))
sns.barplot(x="VehicleType", y="FuelConsumptionRate", data=fuel_consumption)
plt.title("Fuel Consumption Rate by Vehicle Type")
plt.xlabel("Vehicle Type")
plt.ylabel("Fuel Consumption Rate (Liters per Kilometer)")
plt.show()

## Section 3: Product Category Analysis

### Question 1: Contribution to delivery volume and revenue

In [ ]:
query_product_contribution = """
SELECT 
    s.Description AS ProductCategory,
    COUNT(*) AS ShipmentCount,
    SUM(i.TotalAmount) AS Revenue
FROM Shipments s
JOIN Invoices i ON s.ShipmentID = i.ShipmentID
GROUP BY s.Description
ORDER BY Revenue DESC
LIMIT 10;
"""
product_contribution = execute_query(query_product_contribution)

# Visualization
plt.figure(figsize=(10, 6))
sns.barplot(x="Revenue", y="ProductCategory", data=product_contribution)
plt.title("Top Product Categories by Revenue")
plt.xlabel("Revenue")
plt.ylabel("Product Category")
plt.show()

## Section 4: Customer Satisfaction and Retention

### Question 1: Top reasons for customer complaints

In [ ]:
query_customer_complaints = """
SELECT 
    c.CustomerID,
    COUNT(*) AS ComplaintCount
FROM Customers c
JOIN Shipments s ON c.CustomerID = s.CustomerID
WHERE s.Status = 'Cancelled' OR s.Description LIKE '%complaint%'
GROUP BY c.CustomerID
ORDER BY ComplaintCount DESC
LIMIT 10;
"""
customer_complaints = execute_query(query_customer_complaints)

# Visualization
plt.figure(figsize=(10, 6))
sns.barplot(x="ComplaintCount", y="CustomerID", data=customer_complaints)
plt.title("Top Customers with Most Complaints")
plt.xlabel("Number of Complaints")
plt.ylabel("Customer ID")
plt.show()

## Section 5: Cost Optimization

### Question 1: Main cost drivers

In [ ]:
query_cost_drivers = """
SELECT 
    SUM(fc.Cost) AS FuelCost,
    SUM(m.Cost) AS MaintenanceCost
FROM FuelConsumptionRecords fc
JOIN MaintenanceRecords m ON fc.VehicleID = m.VehicleID;
"""
cost_drivers = execute_query(query_cost_drivers)

# Visualization
cost_data = cost_drivers.melt(var_name="CostType", value_name="Cost")
plt.figure(figsize=(8, 6))
sns.barplot(x="CostType", y="Cost", data=cost_data)
plt.title("Main Cost Drivers")
plt.xlabel("Cost Type")
plt.ylabel("Total Cost")
plt.show()